#Búsqueda binaria


## Motivación: ejemplo de programación con invariantes: Calcular $y = x^n$

Supongamos que no tuviéramos una operación de elevación a potencia, y que necesitáramos calcular $x^n$ para $n$ entero no negativo.
El algoritmo obvio es calcular $x*x*\cdots *x$ ($n$ veces):

In [ ]:
def potencia(x, n):
    y=1
    for k in range(0,n):
        y*=x
    return y

In [ ]:
print(potencia(2,10))

1024


El invariante, esto es, lo que se cumple al comenzar cada nueva iteración es $y = x^k$. Así, al inicio, cuando $k=0$, se tiene $y=1$ (inicialización), y al término, cuando $y=n$, se tiene la condición final buscada. La preservación del invariante consiste en multiplicar $y$ por $x$, porque así se sigue cumpliendo el invariante cuando $k$ se incrementa en $1$.

Este algoritmo ejecuta $n$ multiplicaciones para calcular $x^n$ y, si tomamos en cuenta todo lo que hace, es evidente que demora un tiempo proporcional a $n$, lo cual escribiremos $O(n)$ y lo leeremos "del orden de $n$". (Más adelante definiremos precisamente esta notación, y veremos que podríamos ser más precisos todavía al describir el tiempo que demora un algoritmo)

##¿Será posible calcular una potencia de manera más eficiente?

Para ver cómo podríamos mejorar el algoritmo, comenzaremos por reescribirlo de modo que la variable $k$ vaya disminuyendo en lugar de ir aumentando, usando para ello la instrucción `while`:

In [ ]:
def potencia(x, n):
    y=1
    k=n
    while k>0:
        y*=x
        k-=1
    return y

In [ ]:
print(potencia(2,10))

1024


El **invariante** en este caso **sería $y = x^{n-k}$ o**, lo que es lo mismo, $y * x^k = x^n$.

El reescribirlo de esta manera nos permite hacer el siguiente truco: vamos a introducir una variable

$$z$$

cuyo valor inicial es 

$$x$$

 y reformular el invariante como $y * z^k = x^n$ y preservarlo aprovechando que $y*z^k = (y*z)*z^{k-1}$:

In [ ]:
def potencia(x, n):
    y=1
    k=n
    z=x
    while k>0:
        y*=z
        k-=1
    return y

In [ ]:
print(potencia(2,10))

1024


Este cambio podría parecer ocioso, pero gracias a él ahora tenemos un grado adicional de libertad: en efecto, podemos modificar la variable $z$ en la medida que eso no haga que el invariante deje de cumplirse.

En particular, una oportunidad de hacer esto aparece cuando $k$ es par. En ese caso, como $z^n=(z^2)^{n/2}$, si elevamos $z$ al cuadrado y al mismo tiempo dividimos $k$ a la mitad, ambos cambios se complementan para hacer que el invariante se preserve. El algoritmo resultante se llama el ***algoritmo binario***.

In [ ]:
def potencia(x, n):
    y=1
    k=n
    z=x
    while k>0:
        if k%2==0: # caso k par
            z=z*z
            k=k/2
        else:      # caso k impar
            y*=z 
            k-=1
    return y

In [ ]:
print(potencia(2,10))

1024


Este algoritmo admite todavía una pequeña optimización. Cuando $k$ se divide por $2$, no solo se preserva el invariante, sino que además $k$ sigue siendo $>0$, y por lo tanto no es necesario en ese caso volver a preguntar por la condición del `while`. El algoritmo queda como sigue:

In [ ]:
def potencia(x, n):
    y=1
    k=n
    z=x
    while k>0:
        while k%2==0: # caso k par
            z=z*z
            k//=2
        y*=z # aquí estamos seguros que k es impar
        k-=1
    return y

In [ ]:
print(potencia(2,10))

1024


Este algoritmo (en cualquiera de las dos últimas versiones) se llama el *algoritmo binario*, y es mucho más eficiente que el algoritmo inicial. Cada vez que se da el caso par, $k$ disminuye a a mitad, y eso ocurre al menos la mitad de las veces. Pero si $k$ comienza con el valor $n$, la operación de dividir por $2$ se puede ejecutar a lo más $\log_2{n}$ veces, por lo tanto el tiempo total de ejecución es $O(\log_2{n})$, en lugar de $O(n)$. 

Decimos que el algoritmo original era de tiempo lineal, y que el algoritmo binario es de tiempo logarítmico. Para $n$ grande, la diferencia de eficiencia es muy grande en favor del algoritmo binario.

Una observación importante es que para que el algoritmo binario funcione, solo es necesario que $x$ $y$, $z$ pertenezcan a un conjunto para el cual hay definida una operación multiplicativa que sea asociativa y que tenga un elemento neutro. Por lo tanto, este algoritmo no solo sirve para elevar a potencia números enteros o reales, sino que además, por ejemplo, para calcular potencias de *matrices*. 

## Ejemplo de programación con invariantes: Evaluación de un polinomio

Supongamos que se tiene un polinomio

$$
P(x) = \sum_{0<=k<=n}{a_k x^k}
$$

y se desea calcular su valor en un punto dado $x$.

Una solución trivial se puede obtener directamente de la fórmula anterior:

In [ ]:
def evalp(a,x):
    """Evalúa en el punto x el polinomio cuyos coeficientes son a[0], a[1],...
    Retorna el valor calculado
    """
    P=0
    for k in range(0,len(a)):
        # Invariante: P=a[0]+a[1]*x+...+a[k-1]*x**(k-1)
        P += a[k]*x**k
    return P

Podemos probar esta función evaluando el polinomio

$$
P(x) = 5+2x-3x^2+4x^3
$$

en el punto $x=2$:

In [ ]:
print(evalp([5,2,-3,4],2))

29


El problema es que este algoritmo puede ser ineficiente. Si el sistema calcula ``x**k`` de manera simple, el tiempo total de ejecución sería del orden de $n^2$, y si lo calcula usando el algoritmo binario, el tiempo sería del orden de $n\log{n}$. Veremos que esto se puede reducir a tiempo lineal.

Para esto, vamos a introducir una variable adicional, digamos $y$, que almacene el valor de $x^k$ necesario para cada iteración. Para preservar este invariante, al final de cada vuelta del ciclo debemos dejarla multiplicada por $x$, para que tenga el valor correcto al iniciarse la siguiente iteración:

In [ ]:
def evalp(a,x):
    """Evalúa en el punto x el polinomio cuyos coeficientes son a[0], a[1],...
    Retorna el valor calculado
    """
    P=0
    y=1
    for k in range(0,len(a)):
        # Invariante: P=a[0]+a[1]*x+...+a[k-1]*x**(k-1) and y=x**k
        P += a[k]*y
        y *= x
    return P

In [ ]:
print(evalp([5,2,-3,4],2))

29


---